
# Cyclistic 2022 Data Analysis using BigQuery SQL

This notebook documents the steps taken to prepare and analyze the Cyclistic 2022 bike trip data using Google BigQuery SQL. Below are SQL queries used to create unified tables and perform exploratory data analysis.


## Step 1: Combine all monthly tables into one table

## Step 1: Combine All Monthly Tables
We use the `UNION ALL` command to merge all 12 individual monthly tables into a single table called `all_months`. This simplifies our analysis by allowing us to work with one unified dataset.

In [ ]:

CREATE TABLE awesome-tempo-374012.cyclistic_2022.all_months AS
  SELECT *
  FROM (
    SELECT * FROM `awesome-tempo-374012.cyclistic_2022.January`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.February`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.March`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.April`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.May`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.June`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.July`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.August`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.September`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.October`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.November`
    UNION ALL SELECT * FROM `awesome-tempo-374012.cyclistic_2022.December`
  );


## Step 2: Create Main Table with Derived Columns
We add new fields to help with time-based analysis:
- `month`: to identify seasonality
- `day`: for weekday/weekend patterns
- `hour`: to analyze peak hours
- `duration_mins`: to calculate ride duration in minutes

In [ ]:

CREATE TABLE awesome-tempo-374012.cyclistic_2022.main_table AS
  SELECT
    ride_id,
    rideable_type,
    started_at,
    ended_at,
    ride_length,
    member_casual,
    CASE
      WHEN EXTRACT(MONTH FROM started_at) = 1 THEN "January"
      WHEN EXTRACT(MONTH FROM started_at) = 2 THEN "February"
      WHEN EXTRACT(MONTH FROM started_at) = 3 THEN "March"
      WHEN EXTRACT(MONTH FROM started_at) = 4 THEN "April"
      WHEN EXTRACT(MONTH FROM started_at) = 5 THEN "May"
      WHEN EXTRACT(MONTH FROM started_at) = 6 THEN "June"
      WHEN EXTRACT(MONTH FROM started_at) = 7 THEN "July"
      WHEN EXTRACT(MONTH FROM started_at) = 8 THEN "August"
      WHEN EXTRACT(MONTH FROM started_at) = 9 THEN "September"
      WHEN EXTRACT(MONTH FROM started_at) = 10 THEN "October"
      WHEN EXTRACT(MONTH FROM started_at) = 11 THEN "November"
      ELSE "December"
    END AS month,
    CASE
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 1 THEN "Sunday"
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 2 THEN "Monday"
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 3 THEN "Tuesday"
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 4 THEN "Wednesday"
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 5 THEN "Thursday"
      WHEN EXTRACT(DAYOFWEEK FROM started_at) = 6 THEN "Friday"
      ELSE "Saturday"
    END AS day,
    EXTRACT(HOUR FROM started_at) AS hour,
    DATE_DIFF(ended_at, started_at, minute) AS duration_mins
  FROM `awesome-tempo-374012.cyclistic_2022.all_months`
  ORDER BY started_at ASC;


## Step 3: Membership Proportions
This query shows the total number of rides and the percentage split between casual users and annual members.

In [ ]:

SELECT
  member_casual AS membership_type,
  COUNT(member_casual) AS total_rides,
  ROUND(COUNT(member_casual) / SUM(COUNT(member_casual)) OVER() * 100, 2) AS percentage
FROM 
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY
  member_casual
ORDER BY
  member_casual;


## Step 4: Bike Type Usage by Membership
This gives insight into how different bike types are used by casual riders vs. members.

In [ ]:

SELECT
  rideable_type AS bike_type,
  member_casual AS membership_type,
  COUNT(rideable_type) AS total_rides,
  ROUND(COUNT(rideable_type) / SUM(COUNT(rideable_type)) OVER(PARTITION BY rideable_type) * 100, 2) AS percentage_per_bike_type
FROM 
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY 
  rideable_type, member_casual
ORDER BY
  rideable_type, member_casual;


## Step 5: Monthly Ride Frequency
We analyze ride trends across different months to identify seasonal behaviors and usage patterns.

In [ ]:

SELECT 
  month,
  member_casual,
  COUNT(month) AS total_rides,
  ROUND(COUNT(month) / SUM(COUNT(month)) OVER(PARTITION BY month) * 100, 2) AS percentage_per_month
FROM
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY
  month, member_casual
ORDER BY
  month, member_casual;


## Step 6: Ride Frequency by Day
This query shows the ride counts for each day of the week, broken down by membership type.

In [ ]:

SELECT 
  day,
  member_casual,
  COUNT(day) AS total_rides
FROM
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY
  day, member_casual
ORDER BY
  day, member_casual;


## Step 7: Ride Frequency by Hour
This helps us understand the time of day when casual and member riders are most active.

In [ ]:

SELECT 
  member_casual,
  hour,
  COUNT(hour) AS total_rides
FROM
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY
  member_casual, hour
ORDER BY
  member_casual, hour;


## Step 8: Average Ride Duration by Day
This provides the average duration (in minutes) of rides for each day of the week, segmented by user type.

In [ ]:

SELECT
  day,
  member_casual,
  ROUND(AVG(duration_mins), 2) AS avg_duration_mins
FROM
  `awesome-tempo-374012.cyclistic_2022.main_table`
GROUP BY
  day, member_casual
ORDER BY
  day, member_casual;
